In [1]:
pip install ucimlrepo

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [5]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
auto_mpg = fetch_ucirepo(id=9) 
  
# data (as pandas dataframes) 
X = auto_mpg.data.features 
y = auto_mpg.data.targets 
  
# metadata 
# print(auto_mpg.metadata) 
  
# variable information 
# print(auto_mpg.variables) 

In [30]:
type(auto_mpg.data.features)

pandas.core.frame.DataFrame

In [13]:
#import pandas as pd
pd.DataFrame(dict(auto_mpg))
# .to_csv('../Datasets/Auto_MPG.csv')

In [11]:
type(y)

pandas.core.frame.DataFrame

In [14]:
X.isna().sum()

displacement    0
cylinders       0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
dtype: int64

### Imputing mean and finding best combination of alpha and l1_ratio

In [22]:
from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy='mean').set_output(transform='pandas')
X_imp = si.fit_transform(X)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.3, random_state=24)

In [24]:
from sklearn.linear_model import ElasticNet
ENet = ElasticNet()

### Finding the best combination of alpha and l1_ratio
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score 
alpha=[0.01, 0.1, 0.5, 1, 2.5, 3, 5]
ratios=[0.001, 0.3, 0.5, 0.7, 0.9]
scores = []
for i in alpha :
    for j in ratios :
        e1 = ElasticNet(alpha=i, l1_ratio=j)
        e1.fit(X_train, y_train)
        y_pred = e1.predict(X_test)
        scores.append([i, j, r2_score(y_test, y_pred)])

df_scores=pd.DataFrame(scores, columns=['alpha', 'l1_score', 'r2'])
best_combo = df_scores.sort_values('r2', ascending=False).iloc[0]

best_alpha = best_combo[0]
best_l1 = best_combo[1]
best_r2 = best_combo[2]
print(f'Best alpha : {best_alpha}')
print(f'Best l1_ratio : {best_l1}')
print(f'Best r2 : {best_r2}')

Best alpha : 0.1
Best l1_ratio : 0.9
Best r2 : 0.7634320067045604


### Imputing median and finding best combination of alpha and l1_ratio

In [25]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median').set_output(transform='pandas')
X_imp = imp.fit_transform(X_imp)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.3, random_state=24)
from sklearn.linear_model import ElasticNet
ENet = ElasticNet()
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score 
alpha=[0.01, 0.1, 0.5, 1, 2.5, 3, 5]
ratios=[0.001, 0.3, 0.5, 0.7, 0.9]
scores = []
for i in alpha :
    for j in ratios :
        e1 = ElasticNet(alpha=i, l1_ratio=j)
        e1.fit(X_train, y_train)
        y_pred = e1.predict(X_test)
        scores.append([i, j, r2_score(y_test, y_pred)])

df_scores=pd.DataFrame(scores, columns=['alpha', 'l1_score', 'r2'])
best_combo = df_scores.sort_values('r2', ascending=False).iloc[0]
best_alpha = best_combo[0]
best_l1 = best_combo[1]
best_r2 = best_combo[2]
print(f'Best alpha : {best_alpha}')
print(f'Best l1_ratio : {best_l1}')
print(f'Best r2 : {best_r2}')

Best alpha : 0.1
Best l1_ratio : 0.9
Best r2 : 0.7634320067045604


In [27]:
X.horsepower.mean()

104.46938775510205

In [28]:
X.horsepower.median()

93.5

### Apply `ln(1+y)` to y

In [40]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median').set_output(transform='pandas')
X_imp = imp.fit_transform(X_imp)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.3, random_state=24)
from sklearn.linear_model import ElasticNet
ENet = ElasticNet()
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score 
alpha=[0.01, 0.1, 0.5, 1, 2.5, 3, 5]
ratios=[0.001, 0.3, 0.5, 0.7, 0.9]
scores = []
y_trn_ln = np.log1p(y_train)
for i in alpha :
    for j in ratios :
        e1 = ElasticNet(alpha=i, l1_ratio=j)
        e1.fit(X_train, y_trn_ln)
        y_pred = e1.predict(X_test)
        y_pred = np.expm1(y_pred)
        scores.append([i, j, r2_score(y_test, y_pred)])

df_scores=pd.DataFrame(scores, columns=['alpha', 'l1_score', 'r2'])
best_combo = df_scores.sort_values('r2', ascending=False).iloc[0]
best_alpha = best_combo[0]
best_l1 = best_combo[1]
best_r2 = best_combo[2]
print(f'Best alpha : {best_alpha}')
print(f'Best l1_ratio : {best_l1}')
print(f'Best r2 : {best_r2}')

Best alpha : 0.01
Best l1_ratio : 0.9
Best r2 : 0.8059249991482187
